In [1]:
from ipycanvas import Canvas
import import_ipynb
from graph import Graph
from agent import Agent

importing Jupyter notebook from graph.ipynb
importing Jupyter notebook from agent.ipynb


In [2]:
graph = Graph(edges_to_remove=10)
canvas1 = Canvas(width=300, height=300)
canvas1, crds = graph.draw(canvas1)
canvas1

Canvas(height=300, width=300)

In [3]:
# a = random.randint(0, 24)
# b = random.randint(0, 24)
a = 0
b = 24

agent = Agent(a, b, graph)
agent.move()

print('Initial vertex:', a, '(orange)')
print('Final vertex:', b, '(blue)')
print('Steps:', len(agent.hist))
print('Way:')
for i in agent.hist:
    print(i[0], '->', end=' ')
print(agent.hist[len(agent.hist)-1][1])

canvas2 = Canvas(width=300, height=300)
agent.draw(canvas2, crds)

Initial vertex: 0 (orange)
Final vertex: 24 (blue)
Steps: 10
Way:
0 -> 5 -> 10 -> 15 -> 16 -> 17 -> 18 -> 19 -> 18 -> 23 -> 24


Canvas(height=300, width=300)